In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
pip install textract


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
from operator import index
from pandas._config.config import options

import textract as tx#需要pip install
import pandas as pd
import os

import nltk
import spacy

import nltk
import spacy
import re

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


def do_tfidf(token):
    tfidf = TfidfVectorizer(max_df=0.05, min_df=0.002)#把原始文本转化为tf-id的特征矩阵，为后续的文本相似度计算服务
    #max和min_df是对词条出现频率范围做限定，就是既不要出现频率太高的，也不要出现频率太低的
    words = tfidf.fit_transform(token)#fit_transform()作用是先拟合数据，再对数据进行标准化处理，使得每个维度的特征数据方差为1
    sentence = " ".join(tfidf.get_feature_names())#get_geature_names()可以提取出文本的关键字，按照出现频率来取关键字
    return sentence
# Define english stopwords
stop_words = stopwords.words('english')

# load the spacy module and create a nlp object
# This need the spacy en module to be present on the system.
nlp = spacy.load('en_core_web_sm')#加载语言模型
#en_core_web_sm是隶属于spacy的预先训练好的统计模型，类似于一个已经训练好的机器学习模型，用于预测语言特征
# proces to remove stopwords form a file, takes an optional_word list
# for the words that are not present in the stop words but the user wants them deleted.


def remove_stopwords(text, stopwords=stop_words, optional_params=False, optional_words=[]):
    if optional_params:
        stopwords.append([a for a in optional_words])
    return [word for word in text if word not in stopwords]


def tokenize(text):
    # Removes any useless punctuations from the text
    text = re.sub(r'[^\w\s]', '', text)#正则，把不是字母和空白符的都删除掉
    return word_tokenize(text)


def lemmatize(text):
    # the input to this function is a list
    str_text = nlp(" ".join(text))#增添空格
    lemmatized_text = []
    for word in str_text:
        lemmatized_text.append(word.lemma_)
    return lemmatized_text

# internal fuction, useless right now.


def _to_string(List):
    # the input parameter must be a list
    string = " "
    return string.join(List)


def remove_tags(text, postags=['PROPN', 'NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    Takes in Tags which are allowed by the user and then elimnates the rest of the words
    based on their Part of Speech (POS) Tags.
    """
    filtered = []
    str_text = nlp(" ".join(text))
    for token in str_text:
        if token.pos_ in postags:#.pos_输出单词词性
            filtered.append(token.text)#.text是转换成文本
    return filtered


try:
    nlp = spacy.load('en_core_web_sm')#创建一个nlp对象

except ImportError:
    print("Spacy's English Language Modules aren't present \n Install them by doing \n python -m spacy download en_core_web_sm")


def _base_clean(text):
    """
    Takes in text read by the parser file and then does the text cleaning.
    """
    text = tokenize(text)#拆成单词，其实就是word_tokenize()
    text1 = remove_stopwords(text)#移除终止词
    text2 = remove_tags(text1)#移除数字等内容
    text3 = lemmatize(text2)#没什么大用
    return text3


def _reduce_redundancy(text):
    """
    Takes in text that has been cleaned by the _base_clean and uses set to reduce the repeating words
    giving only a single word that is needed.
    """
    return list(set(text))#删除掉text中重复出现的内容（乱序），再放入列表


def _get_target_words(text):
    """
    Takes in text and uses Spacy Tags on it, to extract the relevant Noun, Proper Noun words that contain words related to tech and JD.

    """
    target = []
    sent = " ".join(text)#给text单词之间加入空格
    doc = nlp(sent)
    for token in doc:
        if token.tag_ in ['NN', 'NNP']:#.tag_用法和nltk.pos_tag类似，都是查看其单词属性
            target.append(token.text)#只appendNN和NNP词性的,NN代表单数名词，NNP代表单数的专有名词
    #print(target)
    return target#输出所有NN和NNP词性组成的这个列表


# https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
# https://towardsdatascience.com/the-best-document-similarity-algorithm-in-2020-a-beginners-guide-a01b9ef8cf05

def Cleaner(text):
    sentence = []#创建空list
    sentence_cleaned = _base_clean(text)#给文本进行拆分，并清理掉终止词等内容
    sentence.append(sentence_cleaned)
    sentence_reduced = _reduce_redundancy(sentence_cleaned)#删除掉重复的单词
    sentence.append(sentence_reduced)
    sentence_targetted = _get_target_words(sentence_reduced)#把处理过的无重复单词中，只保留NN和NNP词性的
    sentence.append(sentence_targetted)
    #print(sentence)
    return sentence#输出的sentence大列表包含三个子列表，第一个是拆分后去掉终止词的内容，第二个是删除重复词的内容，第三个是名词词性的内容

nltk.download('stopwords')

resume_dir = "drive/MyDrive/Colab Notebooks/new_resums/"
job_desc_dir = "drive/MyDrive/Colab Notebooks/new_jobdes/"
resume_names = os.listdir(resume_dir)#获取目标路径下的所有文件夹和文件放到一个list里
job_description_names = os.listdir(job_desc_dir)

document = []#创建一个空的列表


def read_resumes(list_of_resumes, resume_directory):#两个参数分别是包含所有文件的list，以及包含这些文件的文件夹路径
    placeholder = []#在遍历前创建的空列表
    for res in list_of_resumes:#如果res属于目标文件夹的文件名
        temp = []#每次遍历都新建temp列表
        temp.append(res)#把文件list里的内容加到temp这个列表中，每个列表只包含当前res的文件内容，也就是文件名字，容量均为1
        #print(temp)
        text = tx.process(resume_directory+res, encoding='ascii')#把该文件的内容提取出来，空格换行等符号都用text的方式显示，如\n代表换行，参数一是文件路径，参数二是编码方式
        text = str(text, 'utf-8')#将对象text转换成人能阅读的方式，就是变成和word文件中几乎一样的内容
        #utf-8是告诉电脑字符串中可能存在中文，电脑可以进行识别
        temp.append(text)#把内容增添到temp中，填到文件名字后面，等于多了一个字符串
        placeholder.append(temp)#把每次循环得到的temp列表存放到总的placeholder列表中封存
    return placeholder#最终封存的是一个大列表，列表元素是一个个小列表，每个列表存放了一个word文件的名称+里面的内容，中间用逗号隔开，等于分别是一个字符串

def read_jobdescriptions(job_description_names, job_desc_dir):
    placeholder = []
    for tes in job_description_names:
        temp = []
        temp.append(tes)
        text = tx.process(job_desc_dir+tes, encoding='ascii')
        text = str(text, 'utf-8')
        temp.append(text)
        placeholder.append(temp)
    return placeholder
document = read_resumes(resume_names, resume_dir)#有问题，把其中的pdf文件删除掉就不报错了。document有两个子列表，分别是名称+内容
#print(document)

def get_cleaned_words(document):
    for i in range(len(document)):#若共有10个文件，len(document)就是10，range（）的结果则为0-9
        raw = Cleaner(document[i][1])#raw取的是document列表中第i个文件的内容进行数据清洗（0是文件名，1是内容）
        #print(raw)
        #print(raw[0])
        print(document[i])#document【i】就是第i个文件的名称+内容
        document[i].append(" ".join(raw[0]))#raw[0]就是去除终止词的内容
        document[i].append(" ".join(raw[1]))#raw【1】就是去掉重复词的内容
        document[i].append(" ".join(raw[2]))#raw【2】就是名词词性的内容
        sentence = do_tfidf(document[i][3].split(" "))#提取关键词，目前选取区间按单词出现频率划分，在0.002-0.05之间认为是关键词
        #该频率区间可以安装经验或结果进行调整
        document[i].append(sentence)
    return document


Doc = get_cleaned_words(document)
#print(document)

Database = pd.DataFrame(document, columns=[
                        "Name", "Context", "Cleaned", "Selective", "Selective_Reduced", "TF_Based"])

Database.to_csv("Resume_Data.csv", index=False)

# Database.to_json("Resume_Data.json", index=False)





job_document = read_jobdescriptions(job_description_names, job_desc_dir)

Jd = get_cleaned_words(job_document)

jd_database = pd.DataFrame(Jd, columns=[
                           "Name", "Context", "Cleaned", "Selective", "Selective_Reduced", "TF_Based"])

jd_database.to_csv("Job_Data.csv", index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['prateek1991121637016637.pdf', "Prateek199112@gmail.com |  |  C: 469-616-6217 | Garland, TX \n\nPRATEEK\n \n\n    YADAV\n \n \n\nEXECUTIVE SUMMARY\n\nDynamic and innovative senior level corporate executive equipped with a wealth of experience\n\nin achieving operational success.  Possessing a progressive aptitude for business process\nmapping, organizational change management, strategic planning, and compliance. Adept at\n\noutperforming aggressive corporate targets and maximizing performances at the lowest possible\ncost. Acting as a major contributor and driver of revenue within various organizations. Skilled in\n\npeople development, with a passion for conceptualizing strategic direction and developing\n\ntactical business plans to stimulate profitable growth. \n\nAREAS OF EXPERTISE\n\nPMP, Operations Leadership, Supply Chain & Logistics, KPI Management, Integrations, SOPs,\n\nContinuous Improvement, Risk\n\nService, Change Management\n\nLogistics, Quality Control\n\nMitigation, P&

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['moizqa80801661508793.pdf', "Name: Moiz Khan                                                                                      \nContact:  (760) 659-0031 \nEmail:  moizqa8080@gmail.com\nCareer Objective:\nA believer of excellence | zeal to up-skill | A multitasking individual | A curious learner and creative \nteam player.\nI personally believe in my capacity and am ready to be molded accordingly, by the firm that employs me.                  \nProfessional Summary\n\uf0b7\n\n8+ years of working experience and expertise in Analysis, Design, Deployment, Distributed and Implementation of \nEnterprise applications in J2EE environment. Comprehensive knowledge of SDLC, having thorough understanding of \nvarious phases like Requirements Analysis, Design, Development and Testing. \n\xa0Extensive project experience in Object Oriented Programming, Data Structures, Algorithms.\n\n\uf0b7\n\uf0b7 Hands-on experience working with Continuous Integration (CI) build-automation tools like Maven, SV

In [15]:
import matplotlib.colors as mcolors
import gensim
import gensim.corpora as corpora
from operator import index
from wordcloud import WordCloud
from pandas._config.config import options
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from PIL import Image
import time

In [16]:
Resumes = pd.read_csv('Resume_Data.csv')
Jobs = pd.read_csv('Job_Data.csv')#直接引用自己的skill des


In [17]:
pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import textdistance as td
#假设A中不重复的单词数量为a，B中不重复的文字数量b，AB共有的文字种类为j

def match(resume, job_des):
    j = td.jaccard.similarity(resume, job_des)#jaccard杰卡特相似度计算j/(a+b-j)
    s = td.sorensen_dice.similarity(resume, job_des)#索雷申相似度计算2j/(a+b)
    c = td.cosine.similarity(resume, job_des)#余弦相似度在向量维度计算AB/AB的行列式
    o = td.overlap.normalized_similarity(resume, job_des)#标准化相似度计算
    total = (j+s+c+o)/4#取上述几种计算相似度的平均值，值域在0-1之间
    # total = (s+o)/2
    return total*100#值域在0-100之间

In [19]:
def calculate_scores_list_1(resumes,job_des):
  scores=[]
  for x in range(job_des.shape[0]):
    score_x=[]
    for y in range(resumes.shape[0]):
      score=match(resumes['TF_Based'][y],job_des['TF_Based'][x])
      score_x.append(score)
    scores.append(score_x)
  return scores

In [20]:
aaa=calculate_scores_list_1(Resumes,Jobs)

In [21]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 116 kB/s 


In [23]:
import kaleido

In [24]:
for i in range(Jobs.shape[0]):
  Resume_loop=Resumes
  Resume_loop['scores']=aaa[i]
  
  Ranked_resumes=Resume_loop.sort_values(by=['scores'],ascending=False).reset_index(drop=True)
  Ranked_resumes['Rank']=pd.DataFrame([i for i in range(1,len(Ranked_resumes['scores'])+1)])
  fig1 = go.Figure(data=[go.Table(
        header=dict(values=["Rank", "Name", "Scores"],
                    fill_color='#00416d',
                    align='center', font=dict(color='white', size=16)),
        cells=dict(values=[Ranked_resumes.Rank, Ranked_resumes.Name, Ranked_resumes.scores],
                   fill_color='#d6e0f0',
                   align='left'))])
  fig1.update_layout(title="Top Ranked Resumes {}".format(Jobs['Name'][i]), width=700, height=1100)##一定记住{}.format()用法
  
  fig1.show()
  fig2 = px.bar(Ranked_resumes,
                  x=Ranked_resumes['Name'], y=Ranked_resumes['scores'], color='scores',
                  color_continuous_scale='haline', title="Score and Rank Distribution")
  fig2.show()